In [1]:
import numpy as np
import pandas as pd
from pyparsing import line
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import scipy.stats as stats

/home/m/m300883/miniconda3/envs/thirdPanel/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import src.Teleconnection.spatial_pattern as ssp
import src.Teleconnection.pattern_statistic as sps
import src.Teleconnection.index_statistic as sis
import src.plots.eof_plots as sept
import src.Teleconnection.temporal_index as sti

In [3]:
import importlib
importlib.reload(sis)
importlib.reload(sti)
importlib.reload(ssp)
importlib.reload(sept)
importlib.reload(sps)

<module 'src.Teleconnection.pattern_statistic' from '/work/mh0033/m300883/3rdPanel/src/Teleconnection/pattern_statistic.py'>

# Data

In [4]:

all_all_dep = xr.open_dataset('/work/mh0033/m300883/3rdPanel/data/indexDiffPattern/all_all_dep.nc').pc
all_first_dep = xr.open_dataset('/work/mh0033/m300883/3rdPanel/data/indexDiffPattern/all_frist_dep.nc').pc
all_last_dep = xr.open_dataset('/work/mh0033/m300883/3rdPanel/data/indexDiffPattern/all_last_dep.nc').pc


# extreme counts

In [10]:
dep_first10_exc,dep_last10_exc = sis.extreme_count([all_all_dep,all_first_dep,all_last_dep])

In [12]:
dep_extc = sis.extr_count_df(dep_first10_exc,dep_last10_exc)

In [13]:
dep_extc

pattern   period  extreme_counts
extr_type mode hlayers                                  
pos       NAO  20000.0      all  first10              18
               20000.0      all   last10              33
               20000.0    first  first10              11
               20000.0    first   last10              20
               20000.0     last  first10              27
...                         ...      ...             ...
neg       EA   100000.0     all   last10              30
               100000.0   first  first10              14
               100000.0   first   last10              16
               100000.0    last  first10              30
               100000.0    last   last10              36

[264 rows x 3 columns]

In [14]:
ea = dep_extc.xs('EA',level='mode')

# first and last10

In [30]:
ea_first = ea[(ea['pattern']=='first' )& (ea['period']=='first10')]

In [28]:
ea_last = ea[(ea['pattern']=='last' )& (ea['period']=='last10')]

In [32]:
ea_first

pattern   period  extreme_counts
extr_type hlayers                                  
pos       20000.0    first  first10              12
          25000.0    first  first10              12
          30000.0    first  first10              14
          40000.0    first  first10              16
          50000.0    first  first10              20
          60000.0    first  first10              21
          70000.0    first  first10              19
          77500.0    first  first10              19
          85000.0    first  first10              20
          92500.0    first  first10              17
          100000.0   first  first10              16
neg       20000.0    first  first10               4
          25000.0    first  first10               6
          30000.0    first  first10               7
          40000.0    first  first10               8
          50000.0    first  first10               9
          60000.0    first  first10              11
          70000.0    first  first10              13
          77500.0    first  first10              14
          85000.0    first  first10              14
          92500.0    first  first10              16
          100000.0   first  first10              14